In [1]:
import pandas as pd


In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('/Users/lucylinderman/Desktop/ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('/Users/lucylinderman/Desktop/ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('/Users/lucylinderman/Desktop/ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

In [4]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
data = pd.merge(pd.merge(ratings, users), movies)

In [8]:
data.head(1)

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama


In [14]:

mean_ratings_genre = data.pivot_table('rating', index='genres', columns='gender', aggfunc='mean')

In [15]:
mean_ratings_genre[:5]

gender,F,M
genres,,
Action,3.367474,3.352991
Action|Adventure,3.701213,3.671115
Action|Adventure|Animation,3.843750,4.217082
Action|Adventure|Animation|Children's|Fantasy,3.146341,2.510638
Action|Adventure|Animation|Horror|Sci-Fi,3.422535,3.563071


In [16]:
top_female_ratings = mean_ratings_genre.sort_values(by='F', ascending=False)

In [17]:
top_female_ratings[:5]

gender,F,M
genres,,
Animation|Comedy|Thriller,4.550802,4.445110
Animation,4.533333,4.353107
Film-Noir|Romance|Thriller,4.448718,4.211073
Sci-Fi|War,4.376623,4.464789
Adventure|Children's|Drama|Musical,4.355030,4.203138


In [18]:
top_male_ratings = mean_ratings_genre.sort_values(by='M', ascending=False)

In [19]:
top_male_ratings[:5]

gender,F,M
genres,,
Sci-Fi|War,4.376623,4.464789
Animation|Comedy|Thriller,4.550802,4.445110
Film-Noir|Mystery,4.319444,4.381536
Adventure|War,4.187970,4.376633
Animation,4.533333,4.353107


In [21]:
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
data['age_group'] = pd.cut(data.age, range(0, 81, 10), right=False, labels=labels)
data[['age', 'age_group']][:10]

,age,age_group
0,1,0-9
1,56,50-59
2,25,20-29
3,25,20-29
4,50,50-59
5,18,10-19
6,1,0-9
7,25,20-29
8,25,20-29
9,45,40-49


In [25]:
mean_ratings_age = data.pivot_table('rating', index='title', columns='age_group', aggfunc='mean')

In [26]:
mean_ratings_age[:5]

age_group,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79
title,,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN,NaN
'Night Mother (1986),2.0,4.666667,3.423077,2.904762,3.833333,3.750000,NaN,NaN
'Til There Was You (1997),3.5,2.500000,2.666667,2.900000,2.333333,2.600000,NaN,NaN
"'burbs, The (1989)",4.5,3.244444,2.652174,2.818182,2.545455,3.100000,NaN,NaN
...And Justice for All (1979),3.0,3.428571,3.724138,3.657143,4.100000,3.674419,NaN,NaN


In [53]:
def same_ratings(user_id, movie_id):
    match_rating = data.loc[(data['movie_id']==movie_id) & (data['rating']==data.loc[user_id, 'rating'])]
    return match_rating

In [52]:
test = same_ratings(1, 1193)
test[:5]

In [71]:
#given a user, find all movies rated 5 by users in the same age bracket and gender
#results = movies loved by users with similar demographics
#removes duplicates
def match_profile(user_id):
    match_profile = data.loc[(data['gender']==data.loc[user_id, 'gender']) & (data['rating']==5) & (data['age_group']==data.loc[user_id, 'age_group'])]
    result = match_profile.drop_duplicates(subset='movie_id')
    return result

In [72]:
example = match_profile(3)
example[:100]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,age_group
16,53,1193,5,977946400,M,25,0,96931,One Flew Over the Cuckoo's Nest (1975),Drama,20-29
1728,53,661,5,977979726,M,25,0,96931,James and the Giant Peach (1996),Animation|Children's|Musical,20-29
2257,53,914,5,977979589,M,25,0,96931,My Fair Lady (1964),Musical|Romance,20-29
2895,36,3408,5,983647331,M,25,3,94123,Erin Brockovich (2000),Drama,20-29
4202,3,2355,5,978298430,M,25,15,55117,"Bug's Life, A (1998)",Animation|Children's|Comedy,20-29
5905,3,1197,5,978297570,M,25,15,55117,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,20-29
8245,204,1287,5,983855193,M,25,7,92123,Ben-Hur (1959),Action|Adventure|Drama,20-29
8929,12,2804,5,978220237,M,25,12,32793,"Christmas Story, A (1983)",Comedy|Drama,20-29
10286,53,594,5,977979520,M,25,0,96931,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,20-29
11044,12,919,5,978220120,M,25,12,32793,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,20-29
